In [ ]:
# !pip3 install kafka-python==1.4.7 streamz==0.5.2 --user

In [ ]:
import os
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [ ]:
os.environ['ENABLE_JSON_LOGGING'] = 'true'
os.environ['LOGLEVEL'] = 'DEBUG'

In [ ]:
import waterhealer as wh
from waterhealer import Stream
import json
import time
import logging

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(processes = False)
client = Client(cluster)

In [ ]:
def json_loads(rows):
    results = []
    for row in rows:
        results.append(json.loads(row))
    return results

def increment_left(rows):
    results = []
    for row in rows:
        row = row.copy()
        row['left'] = row['data'] + 1
        results.append(row)
    return results

def increment_right(rows):
    results = []
    for row in rows:
        row = row.copy()
        row['right'] = row['data'] + 1
        results.append(row)
    return results

def combine(tuples):

    left, right = tuples
    results = []
    for i in range(len(left)):
        logging.info(f"need to combine {left[i]['left'] + right[i]['right']}")
        results.append(left[i]['left'] + right[i]['right'])
        
    return results

In [ ]:
source = Stream()
partition = source.scatter().partition_time(5, checkpoint = True)
s = partition.map(json_loads, checkpoint = True)
left = s.map(increment_left, checkpoint = True)
right = s.map(increment_right, checkpoint = True)
gather = left.zip(right).map(combine, checkpoint = True).buffer(8).gather()

In [ ]:
source.visualize()

In [ ]:
for i in range(10):
    data = {'i': i, 'data': i + 1}
    source.emit(json.dumps(data))
    time.sleep(0.1)

In [ ]:
source.wait()

In [ ]:
source.checkpoint